# Implementacja "Robust global stabilization and disturbance rejection of an underactuated nonholonomic airship"

Link do implementowanego artykułu: https://www.sciencedirect.com/science/article/pii/S1474667016379770?via%3Dihub

Imports

In [490]:
import sympy as sp
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt # ploting results
from scipy.integrate import solve_ivp # solving differential equations
MAT = np.array

Definicja zmiennych  - rozdział II

In [491]:
t = sp.symbols('t') # czas
th = sp.symbols('\\theta') # kąt obrotu sterowca w układzie globalnym
u = sp.symbols('u') # prędkość wzdłużna sterowca w układzie lokalnym
v = sp.symbols('v') # prędkość poprzeczna sterowca w układzie lokalnym
r = sp.symbols('r') # prędkość obrotowa sterowca w układzie globalnym
x, y = sp.symbols('x y') # położenie sterowca w układzie globalnym
z1, z2, z3 = sp.symbols('z_{1} z_{2} z_{3}') # ?
tauu, taur = sp.symbols('\\tau_{u} \\tau_{r}') # sygnały sterujące kolejno: siła ciągu, siła skręcająca 
ub, vb, z1b, z2b, z3b = sp.symbols('\\bar{u} \\bar{v} \\bar{z}_{1} \\bar{z}_{2} \\bar{z}_{3}')#nowe zmienne stanu po transforma

Definicja współczynników i stałych 

In [492]:
Cu, Cv, Cr = sp.symbols('C_{u} C_{v} C_{r}') #współczynniki aerodynamiczne względem osi u v r
Vw = sp.symbols('V_{w}') #prędkość wiatru (w artykule jest zawsze stała)
a_ship, b_ship = sp.symbols('a_{ship} b_{ship}') #długość sterowca, szerokość sterowca
m, m3 = sp.symbols('m m_{3}') # masa strowca m=m1=m2, masa sterowca m3
d1, d2, d3 = sp.symbols('d_{1} d_{2} d_{3}') # współczynniki tłumienia kolejno w osi u v r
d1f, d2f, d3f, th3f = sp.symbols('\\underline{d}_{1} \\underline{d}_{2} \\underline{d}_{3} \\underline{\\theta}_{3}') # dolne wartości wsp tłumienia d1, d2, d3
d1c, d2c, d3c, th3c = sp.symbols('\\bar{d}_{1} \\bar{d}_{2} \\bar{d}_{3} \\bar{\\theta}_{3}') # górne wartości współczynników tłumienia d1, d2, d3
d10, d20, th30 = sp.symbols('d_{10} d_{20} \\theta_{30}') # niepewności
Delta_1, Delta_2, Delta_3, delta_1, delta_2, delta_3 = sp.symbols('\\Delta_{1} \\Delta_{2} \\Delta_{3} \\delta_{1} \\delta_{2} \\delta_{3}')# niepewności
k1, k2, k3, a = sp.symbols('k_{1} k_{2} k_{3} a') # współczynniki wzmocnień syg ster tau_r
k_th, k_r  = sp.symbols('k_{\\theta} k_{r}') # jak wyżej
k1b, k2b, k3b, k4b, k5b = sp.symbols('\\bar{k}_{1} \\bar{k}_{2} \\bar{k}_{3} \\bar{k}_{4} \\bar{k}_{5}')#wsp wzm sterownika tau_u
th_1, th_2, th_3 = sp.symbols('\\theta_1 \\theta_2 \\theta_3') # współczynniki th z obliczone na podst wzoru 4.4
lam_1, lam_2 = sp.symbols('\lambda_1 \lambda_2') # współczynniki lam z obliczone na podst wzoru 4.4
delta_d = sp.symbols('\\Delta_{d}') # współczynnik deltad ze wzoru 4.3
th_0, r_0 = sp.symbols('th(0) r(0)')# wartości początkowe

Definicja funkcji - rozdział II (2.2)

In [493]:
F = sp.sqrt(a_ship**2 * sp.sin(th)**2 + b_ship**2 * sp.cos(th)**2)
Ff = sp.symbols('F', cls=sp.Function)# to będzie funkcja F
fu = Cu * Vw**2 * Ff(th) * sp.cos(th)
fv = Cv * Vw**2 * Ff(th) * sp.sin(th)
fr = Cr * Vw**2 * Ff(th) * sp.cos(2 * th)
#wyświetlanie implementowanych wzorów
#display(fu,fv,fr,F)

Opis modelu kinematycznego, geometrycznego powiązania pomiędzy układem globalnym a układem lokalnym - rozdział II (2.4)

In [494]:
dx = u * sp.cos(th) - v * sp.sin(th)
dy = u * sp.sin(th) + v * sp.cos(th)
dth = r
#wyświetlanie implementowanych wzorów
#display(dx,dy)

Ciąg dalszy opisu kinematyki, z1 z2 - rozdział II (2.5)

In [495]:
#z1 = x * sp.cos(th) + y * sp.sin(th)
#z2 = -x * sp.sin(th) + y * sp.cos(th)
#wyświetlanie implementowanych wzorów
#display(z1,z2)

Równanie stanu uzyskane przez połączenie kinematyki i dynamiki sterowca - rozdział II (2.6)

In [496]:
# układ Sigma1:
A1 = sp.Matrix([[-d1,r,0,0],[-r,-d2,0,0],[1,0,0,r],[0,1,-r,0]]) # macierz stanu
q = sp.Matrix([[u,v,z1,z2]]).T # wektor stanu
b1 = sp.Matrix([[1/m,0,0,0]]).T # macierz sterowania
f = sp.Matrix([[fu/m,fv/m,0,0]]).T # wektor sił zakłóceń - wiatru
dq = A1 @ q + b1 * tauu + f
#wyświetlanie implementowanych wzorów
#display(A1,q,b1,f,dq)

In [497]:
# układ Sigma2:
dr = -d3 * r + (1/m3) * (taur + fr)
dth = r
#wyświetlanie implementowanych wzorów
#display(dr,dth)

Sygnał sterujący dla podsystemu Sigma2 - rozdział IV (4.1)

In [498]:
# k_th = k1 * k2
# k_r = k1 + k2 - d3f
taur = m3 * (-k_r * r - k_th * th + a * sp.exp(-k3 * t) - fr)
#wyświetlanie implementowanych wzorów
#display(taur)

Odpowiedź kąta i prędkości kątowej jest dana - rozdział IV (4.3)

In [499]:
thh = th_1*sp.exp(-lam_1*t) + th_2*sp.exp(-lam_2*t) + th_3*sp.exp(-k3*t) # th(t)
rr = -th_1*lam_1*sp.exp(-lam_1*t) - th_2*lam_2*sp.exp(-lam_2*t) - th_3*k3*sp.exp(-k3*t) # r(t)
tta, rf = sp.symbols('\\theta r', cls=sp.Function) # to będą później funkcje te powyżej
#wyświetlanie implementowanych wzorów
#display(thh,rr)

Implementacja współczynników theta - rozdział IV (4.4)

In [500]:
th_1 = ((lam_2*th_0 + r_0)/(lam_2 - lam_1)) + a/((lam_2 - lam_1)*(k3 - lam_1))
th_2 = ((k1*th_0 + r_0)/(lam_1 - lam_2)) + a/((lam_1 - lam_2)*(k3 - lam_2))
th_3 = a/((lam_1 - k3)*(lam_2 - k3))
#wyświetlanie implementowanych wzorów
#display(th_1,th_2,th_3)

Implementacja współczynników lambda - rozdział IV (4.4)

In [501]:
lam_1 = ((delta_d + k1 + k2) + sp.sqrt(delta_d**2 + 2*delta_d*(k1 + k2) + (k1 - k2)**2))/2
lam_2 = ((delta_d + k1 + k2) - sp.sqrt(delta_d**2 + 2*delta_d*(k1 + k2) + (k1 - k2)**2))/2
delta_d = d3 - d3f
#wyświetlanie implementowanych wzorów
#display(lam_1,lam_2,delta_d)

Implementacja systemu Sigma1e - rozdział IV (4.7)

In [502]:
A2 = sp.Matrix([[-d1,rf(t),0,0,0],[-rf(t),-d2,0,0,0],[1,0,0,rf(t),0],[0,1,-rf(t),0,0],[0,0,1,0,0]])# macierz stanu
b2 = sp.Matrix([[1/m,0,0,0,0]]).T # macierz sterowania
ff = sp.Matrix([[fu/m,-fv/m,0,0,0]]).T # wektor zakłóceń - wiatru
q2 = sp.Matrix([[u,v,z1,z2,z3]]).T # wektor stanu
dX = A2 @ q2 + b2 * tauu + ff
#wyświetlanie implementowanych wzorów
#display(A2,b2,ff,q2,dX)

Wektor stanu po zastosowaniu gładkiej transformacji współrzędnych - rozdział IV (4.9)

In [503]:
ub = u
vb = v * sp.exp(k3 * t)
z1b = z1
z2b = z2 * sp.exp(k3 * t)
z3b = z3

Nowy wektor stanu - rozdział IV (4.10)

In [504]:
Xb = sp.Matrix([[ub,vb,z1b,z2b,z3b]]).T
#wyświetlanie implementowanych wzorów
#display(Xb)

Współczynniki d1, d2, th3 - rozdział IV (4.11)

In [505]:
d1 = d10 + Delta_1
d2 = d20 + Delta_2
th3 = th30 + Delta_3
#wyświetlanie implementowanych wzorów
#display(d1,d2,th3)

Wzory potrzebne do obliczenia d1, d2, th3 - rozdział IV (4.12)

In [506]:
th3f = a/((k3 - k1)*(k3 - k2))
th3c = a/(k3**2 - (d3c + k_r)*k3 + k1*k2)
d10 = (d1c + d1f)/2
d20 = (d2c + d2f)/2
th30 = (th3c + th3f)/2
delta_1 = (d1c - d1f)/2
delta_2 = (d2c - d2f)/2
delta_3 = (th3c - th3f)/2
#wyświetlanie implementowanych wzorów
#display(th3f,th3c,d10,d20,th30,delta_1,delta_2,delta_3)

Rozszerzony system Sigma1e bez zakłócenia f(t) - rozdział IV (4.13) i (4.14)

In [507]:
#macierze o stałych współczynnikach
A10 = sp.Matrix([[-d10,0,0,0,0],[th30*k3,-(d20-k3),0,0,0],[1,0,0,0,0],[0,1,th30*k3,k3,0],[0,0,1,0,0]])
D = sp.Matrix([[1,0,0,0],[0,1,1,0],[0,0,0,0],[0,0,0,1],[0,0,0,0]])
E = sp.Matrix([[-delta_1,0,0,0,0],[0,-delta_2,0,0,0],[k3*delta_3,0,0,0,0],[0,0,k3*delta_3,0,0]])
DELTA = sp.Matrix.diag([Delta_1/delta_1,Delta_2/delta_2,Delta_3/delta_3,Delta_3/delta_3])
#funkcja alpha(t)
alp = -(th_1*lam_1*sp.exp(-(lam_1-k3)*t)+th_2*lam_2*sp.exp(-(lam_2-k3)))
af = sp.symbols('\\alpha', cls=sp.Function)# to będzie później funkcja alpha(t)
#macierz o zmiennych współczynnikach
A2t = sp.Matrix([[0,rf(t)*sp.exp(-k3*t),0,0,0],[af(t),0,0,0,0],[0,0,0,rf(t)*sp.exp(-k3*t),0],[0,0,af(t),0,0],[0,0,0,0,0]])
#wyświetlanie implementowanych wzorów
#display(A10,D,E,DELTA,alp,A2t)

Sygnał sterujący tau_u - rozdział IV (4.21)

In [508]:
tau_u = -(k1b*ub+k2b*vb+k3b*z1b+k4b*z2b+k5b*z3b)
#wyświetlanie implementowanych wzorów
#display(tau_u)

Implementacja systemu - rozdział IV (4.25)

In [509]:
kb = sp.Matrix([[k1b,k2b,k3b,k4b,k5b]]) #wektor wzmocnień sterownika
Ac = A10 - b2@kb + D@DELTA@E
#wektor zakłóceń
fb = sp.Matrix([[Cu*Vw**2*Ff(tta(t))*sp.cos(tta(t))/m,-Cv*Vw**2*sp.exp(k3*t)*Ff(tta(t))*sp.sin(tta(t))/m,0,0,0]]).T
#równanie systemu Sigma1e po dekompozycji na macierz stałą i zależną od czasu
dXb = (Ac + A2t)@Xb + fb
#wyświetlanie implementowanych wzorów
#display(kb,Ac,fb,dXb)

# Podstawienie wartości do zmiennych symbolicznych

In [510]:
params = {a_ship:1.2,b_ship:0.5} #Podstawienie wymiarów sterowca
params.update({Cu:0.42,Cv:0.42,Cr:0.42,Vw:0.1}) #Podstawienie wartości współczynników aerodynamicznych i prędkości wiatru
params.update({m:0.072, m3:0.018})#Podstawienie wartości masy strowca m=m1=m2, masa sterowca m3
params.update({d1f:0.008, d2f:0.029, d3f:0.035})#Podstawienie dolnych wartości współczynników tłumienia
params.update({d1c:0.01, d2c:0.057, d3c:0.05})#Podstawienie górnych wartości współczynników tłumienia
params.update({k1:0.52, k2:0.5, k3:0.35, a:0.15})#Podstawienie wartości wzmocnień kontrolera tau_r
params.update({k1b:1.6, k2b:0.13, k3b:1.0, k4b:0.16, k5b:0.0032})#Podstawienie wartości
params.update({})#Podstawienie wartości
params.update({})#Podstawienie wartości
params.update({})#Podstawienie wartości
params.update({th_1:1,th_2:1,th_3:1,lam_1:1,lam_2:1})#Podstawienie wartości

# Podstawienie wartości do zmiennych symbolicznych

# Definiowanie funkcji zależnych 

In [511]:
Ff = sp.lambdify(th,F.subs(params))#funkcja F(th)
F = sp.lambdify(th,F.subs(params))#powieliłem 2 razy, ponieważ na potrzeby wizualizacji wzorów zadeklarowałem 2 takie funkcje


In [ ]:
m3 = m3.subs(params)
m = m.subs(params)
fu = fu.subs(params)
fv = fv.subs(params)
fr = fr.subs(params)
d1f = d1f.subs(params)
d2f = d2f.subs(params)
d3f = d3f.subs(params)
d1c = d1c.subs(params)
d2c = d2c.subs(params)
d3c = d3c.subs(params)
k1 = k1.subs(params)
k2 = k2.subs(params)
k3 = k3.subs(params)
a = a.subs(params)
k1b = k1b.subs(params)
k2b = k2b.subs(params)
k3b = k3b.subs(params)
k4b = k4b.subs(params)
k5b = k5b.subs(params)